In [5]:
!pip install transformers
!pip install tokenizer
!pip install sentencepiece

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 78.0/78.0 KB 1.5 MB/s eta 0:00:00


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("JDBN/t5-base-fr-qg-fquad")
model = T5ForConditionalGeneration.from_pretrained("JDBN/t5-base-fr-qg-fquad")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

file = "simpleT5.tsv"

df = pd.read_csv(file, sep="\t")
df.head(5)

,Unnamed: 0,Question,Reponse
0,1,Quels sont tes centres d'interets,Je suis interessé par beaucoup de choses. Nous...
1,2,Quels sont tes centres d'interets,Je suis interessé par une grande variete de su...
2,3,Quels sont tes sujets preferes,"Mes sujets preferes incluent la robotique, l'i..."
3,4,Quel est ton numero,Je n'ai pas de numero.
4,5,Quel est ton numero,"5446, c'est mon numero maintenant."


In [3]:
df = df[:80]
print(len(df))

80


In [4]:
task_prefix = "AnswerQuestion:"
input_sequences = list(df["Question"])
encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=255,
    truncation=True,
    return_tensors="pt",
)

TypeError: 'T5Tokenizer' object is not callable

In [ ]:
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask


In [ ]:
# encode the targets
target_encoding = tokenizer(
    list(df["Reponse"]), padding="longest", max_length=255, truncation=True
)
labels = target_encoding.input_ids

In [ ]:
import torch
# replace padding token id's of the labels by -100
labels = torch.tensor(labels)
labels[labels == tokenizer.pad_token_id] = -100

In [ ]:
# forward pass
output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

In [ ]:
output.loss

tensor(3.8660, grad_fn=<NllLossBackward0>)

In [ ]:
input_ids = tokenizer("AnswerQuestion: Conseille moi ", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Qu'est-ce que vous conseille moi?
